# NDVI Example (Microsoft Planetary Computer)

In [ ]:
import sys
import os
import numpy as np
import pystac_client
from pystac.extensions.eo import EOExtension as eo
from osgeo import gdal
import planetary_computer

## Set search parameters and perform search

In [ ]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [12, 50],
            [13, 50],
            [13, 51],
            [12, 51],
            [12, 50],
        ]
    ]
}

In [ ]:
time_of_interest = "2022-07-01/2022-08-01"

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
)
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
)
print(search)
# Check how many items were returned
items = search.item_collection()
print(f"Returned {len(items)} Items")

## Choose least cloudy item

In [ ]:
least_cloudy_item = min(items, key=lambda item: eo.ext(item).cloud_cover)
print(
    f"Choosing {least_cloudy_item.id} from {least_cloudy_item.datetime.date()}"
    f" with {eo.ext(least_cloudy_item).cloud_cover}% cloud cover"
)

## Open bands and read them as float arrays

In [ ]:
red_file = least_cloudy_item.assets["B04"].href
nir_file = least_cloudy_item.assets["B08"].href

red_data = gdal.Open(red_file)
nir_data = gdal.Open(nir_file)

red = red_data.ReadAsArray().astype(float)
nir = nir_data.ReadAsArray().astype(float)

## Calculate NDVI

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
ndvi = (nir - red) / (nir + red)

## Set output filename and shape

In [ ]:
outfile_name = "test_ndvi.tif"
(width, height) = ndvi.shape

## Create driver using output filename, x and y pixels, # of bands, and datatype

In [ ]:
driver = gdal.GetDriverByName('GTiff')

## Create driver using output filename, x and y pixels, # of bands, and datatype

In [ ]:
ndvi_data = driver.Create(outfile_name, width, height, 1, gdal.GDT_Float32)
ndvi_data.GetRasterBand(1).WriteArray(ndvi)

## Obtain and set coordinate reference system and projection information

In [ ]:
geo_transform = red_data.GetGeoTransform()
projection = red_data.GetProjection()
ndvi_data.SetGeoTransform(geo_transform) 
ndvi_data.SetProjection(projection)

## Save the GeoTIFF file

In [ ]:
ndvi_data.SetGeoTransform(geo_transform) 
ndvi_data.SetProjection(projection)
ndvi_data.FlushCache()

## Show NDVI result

In [ ]:
from PIL import Image
img = Image.fromarray(np.uint8((ndvi + 1) ** 3 * 255 / 8) , 'L')
img.resize((800, 800), Image.Resampling.BILINEAR)

## Show visual image as reference

In [ ]:
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp

import numpy as np
from PIL import Image

ds = rasterio.open(least_cloudy_item.assets["visual"].href)
band_data = ds.read()
    
img = Image.fromarray(np.transpose(band_data, axes=[1, 2, 0]))
w = img.size[0]
h = img.size[1]
aspect = w / h
target_w = 800
target_h = (int)(target_w / aspect)
img.resize((target_w, target_h), Image.Resampling.BILINEAR)

**END**